<a href="https://colab.research.google.com/github/ridwaanhall/colab-google/blob/main/test/spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NGL FIX

In [ ]:
import csv
import random
import string
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from uuid import uuid4

class Register:
    def __init__(self, username):
        self.url = "https://ngl.link/api/submit"
        self.game_slugs = [
            "",
            "confessions",
            "3words",
            "tbh",
            "shipme",
            "yourcrush",
            "cancelled",
            "dealbreaker"
        ]
        self.username = username

    def _random_device_id(self):
        return str(uuid4())

    def _random_question(self):
        length = random.randint(30, 250)
        return ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation + " ", k=length))

    def _random_game_slug(self):
        return random.choice(self.game_slugs)

    def _generate_form_data(self):
        device_id = self._random_device_id()
        question = self._random_question()
        game_slug = self._random_game_slug()

        return {
            "username": self.username,
            "question": "free palestine. go to hell israel. IDF (israel dance force)",
            "deviceId": device_id,
            "gameSlug": game_slug,
            "referrer": ""
        }

    def _submit_form(self, data):
        response = requests.post(self.url, json=data)
        return response

    def send_message(self):
        data = self._generate_form_data()
        response = self._submit_form(data)

    def send_messages(self, count, batch_size):
        for i in range(0, count, batch_size):
            current_batch_size = min(batch_size, count - i)
            with ThreadPoolExecutor(max_workers=10) as executor:
                futures = [executor.submit(self.send_message) for _ in range(current_batch_size)]
                for _ in tqdm(as_completed(futures), total=current_batch_size, desc=f"Sending messages batch {i // batch_size + 1}"):
                    pass

if __name__ == "__main__":
    # url = "https://ngl.link/api/submit"
    # username = input("username: ")
    username = "yonipi75"
    count_input = input("count (default: 999999999999999): ")
    count = int(count_input) if count_input else 999999999999999
    batch_size_input = input("batch size (default: 1000): ")
    batch_size = int(batch_size_input) if batch_size_input else 1000

    Register(username).send_messages(count, batch_size)

count (default: 999999999999999): 
batch size (default: 1000): 


Sending messages batch 18:  24%|██▍       | 242/1000 [00:16<00:50, 15.08it/s]


KeyboardInterrupt: 

In [ ]:
import os
import random
import string
import time
import requests
from fake_useragent import UserAgent

class RequestSender:
    def __init__(self, url):
        self.url = url

    def send_request(self, username, question, deviceId, gameSlug='', referrer=''):
        headers = self._generate_headers(username)
        data = {
            'username': username,
            'question': question,
            'deviceId': deviceId,
            'gameSlug': gameSlug,
            'referrer': referrer
        }
        return requests.post(self.url, headers=headers, data=data)

    def send_request_with_retry(self, username, question, deviceId, gameSlug='', referrer='', max_retries=3):
        for retry in range(max_retries):
            response = self.send_request(username, question, deviceId, gameSlug, referrer)
            if response.status_code == 429:
                retry_after = int(response.headers.get('Retry-After', 10))
                print(f'Retrying in {retry_after} seconds...', end='', flush=True)
                time.sleep(retry_after)
            elif response.status_code == 200:
                return response
            else:
                print(f"Unexpected error: {response.status_code} - {response.reason}")
                return None
        print("Max retries reached. Failed to send request.")
        return None

    def _generate_headers(self, username):
        sec_ch_ua = [
            '"Microsoft Edge";v="123"',
            '"Not:A-Brand";v="8"',
            '"Chromium";v="123"'
        ]
        sec_ch_ua_mobile = ['?0', '?1']
        sec_ch_ua_platform = ['"Windows"', '"Linux"', '"Macintosh"', '"Android"', '"iOS"']

        return {
            'Accept': '*/*',
            'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'Dnt': '1',
            'Referer': f'https://ngl.link/{username}',
            'Sec-Ch-Ua': random.choice(sec_ch_ua),
            'Sec-Ch-Ua-Mobile': random.choice(sec_ch_ua_mobile),
            'Sec-Ch-Ua-Platform': random.choice(sec_ch_ua_platform),
            'User-Agent': UserAgent().random
        }

def generate_deviceId():
    return '-'.join(
        ''.join(random.choices(string.ascii_lowercase + string.digits, k=length))
        for length in [8, 4, 4, 4, 12]
    )

game_slugs = {
    "1": "",
    "2": "confessions",
    "3": "3words",
    "4": "tbh",
    "5": "shipme",
    "6": "yourcrush",
    "7": "cancelled",
    "8": "dealbreaker"
}

def select_game_slug():
    print("\nSelect a game slug:")
    for key, value in game_slugs.items():
        print(f"{key}: {value}")
    while True:
        choice = input("\nEnter your choice (1-8): ")
        if choice in game_slugs:
            return game_slugs[choice]
        print("Invalid choice. Please enter a number between 1 and 8.")

if __name__ == "__main__":
    url = "https://ngl.link/api/submit"
    request_sender = RequestSender(url)
    username = "yonipi75".lower()

    spam_choice = input("\nDo you want to spam? (yes/no): ").lower()
    if spam_choice in ["yes", "y", ""]:
        spam_count = input("How many times do you want to spam? (default: 10000000): ").strip()
        spam_count = int(spam_count) if spam_count.isdigit() else 10000000

        for i in range(spam_count):
            gameSlug = ""
            deviceId = generate_deviceId()
            message = " "
            referrer = ""
            response = request_sender.send_request_with_retry(username, message, deviceId, gameSlug, referrer)
            print(f'\n{i + 1} of {spam_count}')
            print(f'{response.status_code} {response.reason} = {response.text}')
    else:
        gameSlug = select_game_slug()
        deviceId = generate_deviceId()
        referrer = ""
        message = " "
        response = request_sender.send_request_with_retry(username, message, deviceId, gameSlug, referrer)
        print(f'gameSlug: {gameSlug}')
        print(f'message : {message}')
        print(f'{response.status_code} {response.reason} = {response.text}')



Do you want to spam? (yes/no): no

Select a game slug:
1: 
2: confessions
3: 3words
4: tbh
5: shipme
6: yourcrush
7: cancelled
8: dealbreaker

Enter your choice (1-8): 8
gameSlug: dealbreaker
message :  
200 OK = {"questionId":"cEQUivR0eetD4oQv2jQI","userRegion":"IL"}


## website

In [ ]:
import csv
import random
import string
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

class Register:
    def __init__(self, url, csv_file):
        self.url = url
        self.csv_file = csv_file
        self._setup_csv()

    def _setup_csv(self):
        with open(self.csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["phone", "password", "membership", "email"])

    def _random_phone(self):
        return '08' + ''.join(random.choices(string.digits, k=random.randint(7, 11)))

    def _random_email(self, phone):
        prefix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
        domain = ''.join(random.choices(string.ascii_lowercase, k=3))
        return f"{phone}@{prefix}.{domain}"

    def _random_membership(self):
        return random.choice(["Exclusive", "Basic", "Ultimate", "Premium"])

    def _generate_form_data(self):
        phone = self._random_phone()
        email = self._random_email(phone)
        password = phone
        membership = self._random_membership()

        return {
            "name": phone,
            "phone": phone,
            "email": email,
            "password": password,
            "password_repeat": password,
            "membership": membership,
            "pengundang": "08939470782187"
        }

    def _save_to_csv(self, data):
        with open(self.csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([data['phone'], data['password'], data['membership'], data['email']])

    def _submit_form(self, data):
        response = requests.post(self.url, data=data)
        return response

    def send_message(self):
        data = self._generate_form_data()
        response = self._submit_form(data)

        if response.status_code == 200:
            self._save_to_csv(data)

    def send_messages(self, count, batch_size=128):
        for i in range(0, count, batch_size):
            current_batch_size = min(batch_size, count - i)
            with ThreadPoolExecutor(max_workers=10) as executor:
                futures = [executor.submit(self.send_message) for _ in range(current_batch_size)]
                for _ in tqdm(as_completed(futures), total=current_batch_size, desc=f"Sending messages batch {i // batch_size + 1}"):
                    pass

if __name__ == "__main__":
    register = Register("https://member.digitalsmartcenter.com/auth/proses_register", "passwords_1.csv")
    register.send_messages(128000000000)

## ngl

### 1

In [ ]:
import csv
import random
import string
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from uuid import uuid4

class Register:
    def __init__(self, url, csv_file):
        self.url = url
        self.csv_file = csv_file
        self.game_slugs = [
            "",
            "confessions",
            "3words",
            "tbh",
            "shipme",
            "yourcrush",
            "cancelled",
            "dealbreaker"
        ]
        self._setup_csv()

    def _setup_csv(self):
        with open(self.csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["username", "question", "deviceId", "gameSlug"])

    def _random_device_id(self):
        return str(uuid4())

    def _random_question(self):
        length = random.randint(10, 100)
        return ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation + " ", k=length))

    def _random_game_slug(self):
        return random.choice(self.game_slugs)

    def _generate_form_data(self):
        username = "fictcion" # "rookiecoock" # attazch
        device_id = self._random_device_id()
        question = self._random_question()
        game_slug = self._random_game_slug()

        return {
            "username": username,
            "question": question,
            "deviceId": device_id,
            "gameSlug": game_slug,
            "referrer": ""
        }

    def _save_to_csv(self, data):
        with open(self.csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([data['username'], data['question'], data['deviceId'], data['gameSlug']])

    def _submit_form(self, data):
        response = requests.post(self.url, json=data)
        return response

    def send_message(self):
        data = self._generate_form_data()
        response = self._submit_form(data)

        if response.status_code == 200:
            self._save_to_csv(data)

    def send_messages(self, count, batch_size=1024):
        for i in range(0, count, batch_size):
            current_batch_size = min(batch_size, count - i)
            with ThreadPoolExecutor(max_workers=10) as executor:
                futures = [executor.submit(self.send_message) for _ in range(current_batch_size)]
                for _ in tqdm(as_completed(futures), total=current_batch_size, desc=f"Sending messages batch {i // batch_size + 1}"):
                    pass

if __name__ == "__main__":
    register = Register("https://ngl.link/api/submit", "ngl.csv")
    register.send_messages(99999999999999999993)


In [ ]:
# 08939470782187

### 2

In [ ]:
import csv
import random
import string
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from uuid import uuid4

class Message:
    def __init__(self, username, question, device_id, game_slug):
        self.username = username
        self.question = question
        self.device_id = device_id
        self.game_slug = game_slug

    def to_dict(self):
        return {
            "username": self.username,
            "question": self.question,
            "deviceId": self.device_id,
            "gameSlug": self.game_slug,
            "referrer": ""
        }

class CSVHandler:
    def __init__(self, csv_file):
        self.csv_file = csv_file
        self._setup_csv()

    def _setup_csv(self):
        with open(self.csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["username", "question", "deviceId", "gameSlug"])

    def save_message(self, message):
        with open(self.csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([message.username, message.question, message.device_id, message.game_slug])

class MessageGenerator:
    def __init__(self):
        self.game_slugs = [
            "",
            "confessions",
            "3words",
            "tbh",
            "shipme",
            "yourcrush",
            "cancelled",
            "dealbreaker"
        ]

    def _random_device_id(self):
        return str(uuid4())

    def _random_question(self):
        length = random.randint(10, 100)
        return ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation + " ", k=length))

    def _random_game_slug(self):
        return random.choice(self.game_slugs)

    def generate_message(self, username):
        device_id = self._random_device_id()
        question = self._random_question()
        game_slug = self._random_game_slug()
        return Message(username, question, device_id, game_slug)

class APIClient:
    def __init__(self, url):
        self.url = url

    def submit_message(self, message):
        response = requests.post(self.url, json=message.to_dict())
        return response

class Register:
    def __init__(self, api_client, csv_handler, message_generator):
        self.api_client = api_client
        self.csv_handler = csv_handler
        self.message_generator = message_generator

    def send_message(self, username):
        message = self.message_generator.generate_message(username)
        response = self.api_client.submit_message(message)

        if response.status_code == 200:
            self.csv_handler.save_message(message)

    def send_messages(self, usernames, batch_size=1024):
        total_messages = len(usernames)
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(self.send_message, username) for username in usernames]
            for _ in tqdm(as_completed(futures), total=total_messages, desc="Sending messages"):
                pass

if __name__ == "__main__":
    api_client = APIClient("https://ngl.link/api/submit")
    csv_handler = CSVHandler("ngl.csv")
    message_generator = MessageGenerator()
    register = Register(api_client, csv_handler, message_generator)

    usernames = ["ezzahnor", "rookiecoock", "attazch", "ridwaanhall1"]  # Add more usernames as needed
    register.send_messages(usernames)

Sending messages: 100%|██████████| 4/4 [00:01<00:00,  3.01it/s]


### 3

with LLM

In [ ]:
import csv
import random
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from uuid import uuid4
from transformers import pipeline

class QuestionGenerator:
    def __init__(self):
        self.generator = pipeline('text-generation', model='gpt2')

    def generate_question(self):
        prompt = "Ask an interesting question about someone's life, preferences, or experiences."
        question = self.generator(prompt, max_length=100, num_return_sequences=1)[0]['generated_text']
        return question.strip()

class Message:
    def __init__(self, username, question, device_id, game_slug):
        self.username = username
        self.question = question
        self.device_id = device_id
        self.game_slug = game_slug

    def to_dict(self):
        return {
            "username": self.username,
            "question": self.question,
            "deviceId": self.device_id,
            "gameSlug": self.game_slug,
            "referrer": ""
        }

class CSVHandler:
    def __init__(self, csv_file):
        self.csv_file = csv_file
        self._setup_csv()

    def _setup_csv(self):
        with open(self.csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["username", "question", "deviceId", "gameSlug"])

    def save_message(self, message):
        with open(self.csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([message.username, message.question, message.device_id, message.game_slug])

class APIClient:
    def __init__(self, url):
        self.url = url

    def submit_message(self, message):
        response = requests.post(self.url, json=message.to_dict())
        return response

class Register:
    def __init__(self, api_client, csv_handler, question_generator):
        self.api_client = api_client
        self.csv_handler = csv_handler
        self.question_generator = question_generator
        self.game_slugs = ["", "confessions", "3words", "tbh", "shipme", "yourcrush", "cancelled", "dealbreaker"]

    def _random_device_id(self):
        return str(uuid4())

    def _random_game_slug(self):
        return random.choice(self.game_slugs)

    def send_message(self, username):
        device_id = self._random_device_id()
        game_slug = self._random_game_slug()
        question = self.question_generator.generate_question()
        message = Message(username, question, device_id, game_slug)

        response = self.api_client.submit_message(message)

        if response.status_code == 200:
            self.csv_handler.save_message(message)

    def send_messages(self, usernames, batch_size=32):
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(self.send_message, username) for username in usernames]
            for _ in tqdm(as_completed(futures), total=len(usernames), desc="Sending messages"):
                pass

if __name__ == "__main__":
    api_client = APIClient("https://ngl.link/api/submit")
    csv_handler = CSVHandler("ngl.csv")
    question_generator = QuestionGenerator()
    register = Register(api_client, csv_handler, question_generator)

    usernames = ["ezzahnor", "rookiecoock", "attazch"]  # Add more usernames as needed
    register.send_messages(usernames)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Sending messages:   0%|          | 0/3 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Sending messages: 100%|██████████| 3/3 [00:21<00:00,  7.11s/it]


### 4. llm 1000

In [ ]:
import csv
import random
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from uuid import uuid4
from transformers import pipeline

class QuestionGenerator:
    def __init__(self):
        self.generator = pipeline('text-generation', model='gpt2')

    def generate_question(self):
        prompt = "Ask an interesting question about someone's life, preferences, or experiences."
        question = self.generator(prompt, max_length=100, num_return_sequences=1)[0]['generated_text']
        return question.strip()

class Message:
    def __init__(self, username, question, device_id, game_slug):
        self.username = username
        self.question = question
        self.device_id = device_id
        self.game_slug = game_slug

    def to_dict(self):
        return {
            "username": self.username,
            "question": self.question,
            "deviceId": self.device_id,
            "gameSlug": self.game_slug,
            "referrer": ""
        }

class CSVHandler:
    def __init__(self, csv_file):
        self.csv_file = csv_file
        self._setup_csv()

    def _setup_csv(self):
        with open(self.csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["username", "question", "deviceId", "gameSlug"])

    def save_message(self, message):
        with open(self.csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([message.username, message.question, message.device_id, message.game_slug])

class APIClient:
    def __init__(self, url):
        self.url = url

    def submit_message(self, message):
        response = requests.post(self.url, json=message.to_dict())
        return response

class Register:
    def __init__(self, api_client, csv_handler, question_generator):
        self.api_client = api_client
        self.csv_handler = csv_handler
        self.question_generator = question_generator
        self.game_slugs = ["", "confessions", "3words", "tbh", "shipme", "yourcrush", "cancelled", "dealbreaker"]

    def _random_device_id(self):
        return str(uuid4())

    def _random_game_slug(self):
        return random.choice(self.game_slugs)

    def send_message(self, username):
        device_id = self._random_device_id()
        game_slug = self._random_game_slug()
        question = self.question_generator.generate_question()
        message = Message(username, question, device_id, game_slug)

        response = self.api_client.submit_message(message)

        if response.status_code == 200:
            self.csv_handler.save_message(message)

    def send_messages(self, usernames, count_per_user, batch_size=32):
        total_messages = len(usernames) * count_per_user
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = []
            for _ in range(count_per_user):
                futures.extend(executor.submit(self.send_message, username) for username in usernames)
            for _ in tqdm(as_completed(futures), total=total_messages, desc="Sending messages"):
                pass

if __name__ == "__main__":
    api_client = APIClient("https://ngl.link/api/submit")
    csv_handler = CSVHandler("ngl.csv")
    question_generator = QuestionGenerator()
    register = Register(api_client, csv_handler, question_generator)

    usernames = ["ezzahnor", "rookiecoock", "attazch"]
    count_per_user = 1000
    register.send_messages(usernames, count_per_user)

### 5. llm fixed batch size

In [ ]:
import csv
import random
import requests
import string
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from uuid import uuid4

class QuestionGenerator:
    def generate_questions(self, count):
        questions = []
        for _ in range(count):
            length = random.randint(10, 100)
            question = ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation + " ", k=length))
            questions.append(question)
        return questions

class Message:
    def __init__(self, username, question, device_id, game_slug):
        self.username = username
        self.question = question
        self.device_id = device_id
        self.game_slug = game_slug

    def to_dict(self):
        return {
            "username": self.username,
            "question": self.question,
            "deviceId": self.device_id,
            "gameSlug": self.game_slug,
            "referrer": "https://t.co/"
        }

class CSVHandler:
    def __init__(self, csv_file):
        self.csv_file = csv_file
        self._setup_csv()

    def _setup_csv(self):
        with open(self.csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["username", "question", "deviceId", "gameSlug"])

    def save_messages(self, messages):
        with open(self.csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerows([[msg.username, msg.question, msg.device_id, msg.game_slug] for msg in messages])

class APIClient:
    def __init__(self, url):
        self.url = url

    def submit_messages(self, messages):
        payload = [msg.to_dict() for msg in messages]
        try:
            response = requests.post(self.url, json=payload)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            if response is not None:
                print(f"Response content: {response.content}")
            return None

class Register:
    def __init__(self, api_client, csv_handler, question_generator):
        self.api_client = api_client
        self.csv_handler = csv_handler
        self.question_generator = question_generator
        self.game_slugs = ["", "confessions", "3words", "tbh", "shipme", "yourcrush", "cancelled", "dealbreaker"]

    def _random_device_id(self):
        return str(uuid4())

    def _random_game_slug(self):
        return random.choice(self.game_slugs)

    def send_messages(self, usernames, count_per_user, batch_size=100):
        total_messages = len(usernames) * count_per_user

        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = []
            for username in usernames:
                questions = self.question_generator.generate_questions(count_per_user)
                messages = [Message(username, q, self._random_device_id(), self._random_game_slug()) for q in questions]
                chunks = list(self._chunk_messages(messages, batch_size))

                for batch in chunks:
                    futures.append(executor.submit(self.api_client.submit_messages, batch))

                # Save messages to CSV after each user
                self.csv_handler.save_messages(messages)

            for _ in tqdm(as_completed(futures), total=len(futures), desc="Sending messages"):
                pass

    def _chunk_messages(self, messages, batch_size):
        for i in range(0, len(messages), batch_size):
            yield messages[i:i + batch_size]

if __name__ == "__main__":
    api_client = APIClient("https://ngl.link/api/submit")
    csv_handler = CSVHandler("ngl.csv")
    question_generator = QuestionGenerator()
    register = Register(api_client, csv_handler, question_generator)

    usernames = ["ezzahnor", "rookiecoock", "attazch", "ridwaanhall1"]
    count_per_user = 1000
    batch_size = 100
    register.send_messages(usernames, count_per_user, batch_size)


,username,question,deviceId,gameSlug
0,ezzahnor,">^CBl""2YI~^c*OJYEI'VL.QRVbta\/B$VTiK.LR5qU\JJv...",9d58a97d-bb60-48ca-88e7-27f8ec69dbea,3words
1,rookiecoock,"rL:1^),Jw8d-(d^GvBr_qjoI8qxEMr;^^[=&:\m<HO=D;D...",c1fe1ac7-4739-4e09-8c0c-5ba881b9660a,tbh
2,attazch,DUI=I;0[96=P)AHnaaZ%iT.QU%iYIH v`=ryp|rY]`sV$s...,183c8635-2a6d-41ed-89d6-6f2b19dc237d,confessions
3,ridwaanhall1,K.g1`~:8H4EwOI{W2G>firDYZGESN-0A<%mSeee 2;Ua0y...,b81b4dfe-5fc3-4677-a428-0277e949f18e,yourcrush


## ngl testing new

### 1

In [ ]:
import csv
import random
import string
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from uuid import uuid4

class Register:
    def __init__(self, url, csv_file):
        self.url = url
        self.csv_file = csv_file
        self.game_slugs = [
            "",
            "confessions",
            "3words",
            "tbh",
            "shipme",
            "yourcrush",
            "cancelled",
            "dealbreaker"
        ]
        self._setup_csv()

    def _setup_csv(self):
        with open(self.csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["username", "question", "deviceId", "gameSlug"])

    def _random_device_id(self):
        return str(uuid4())

    def _random_question(self):
        length = random.randint(10, 100)
        return ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation + " ", k=length))

    def _random_game_slug(self):
        return random.choice(self.game_slugs)

    def _generate_form_data(self):
        username = "aluradrafts"
        device_id = self._random_device_id()
        question = self._random_question()
        game_slug = self._random_game_slug()

        return {
            "username": username,
            "question": question,
            "deviceId": device_id,
            "gameSlug": game_slug,
            "referrer": ""
        }

    def _save_to_csv(self, data):
        with open(self.csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([data['username'], data['question'], data['deviceId'], data['gameSlug']])

    def _submit_form(self, data):
        response = requests.post(self.url, json=data)
        return response

    def send_message(self):
        data = self._generate_form_data()
        response = self._submit_form(data)

        if response.status_code == 200:
            self._save_to_csv(data)

    def send_messages(self, count, batch_size=1024):
        for i in range(0, count, batch_size):
            current_batch_size = min(batch_size, count - i)
            with ThreadPoolExecutor(max_workers=10) as executor:
                futures = [executor.submit(self.send_message) for _ in range(current_batch_size)]
                for _ in tqdm(as_completed(futures), total=current_batch_size, desc=f"Sending messages batch {i // batch_size + 1}"):
                    pass

if __name__ == "__main__":
    register = Register("https://ngl.link/api/submit", "ngl.csv")
    register.send_messages(99999999999999999993)

Sending messages batch 27:  10%|▉         | 101/1024 [00:05<00:47, 19.54it/s]

## secreto

In [ ]:
import requests
import random
import string
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

class Register:
    def __init__(self, url):
        self.url = url

    def _random_message(self):
        length = random.randint(10, 100)
        return ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation + " ", k=length))

    def _generate_form_data(self):
        return {
            "id": "12749653_7l9p1",
            "message": self._random_message()*1000
        }

    def _submit_form(self, data):
        response = requests.post(self.url, json=data)
        return response

    def send_messages(self, count, batch_size=100):
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(self.send_message) for _ in range(count)]
            for _ in tqdm(as_completed(futures), total=count, desc="Sending messages"):
                pass

    def send_message(self):
        message_count = 10  # Change this to the desired number of messages per batch
        for _ in range(message_count):
            data = self._generate_form_data()
            response = self._submit_form(data)

if __name__ == "__main__":
    url = "https://api.secreto.site/sendmsg"
    register = Register(url)
    register.send_messages(count=1000, batch_size=100)


Sending messages:  25%|██▌       | 250/1000 [06:35<19:47,  1.58s/it]


KeyboardInterrupt: 